#### Setup

In [40]:
# standard imports 
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import optim
from ipdb import set_trace

# jupyter setup
%matplotlib inline
%load_ext autoreload
%autoreload 2

# own modules
from dataloader import CAL_Dataset
from net import get_model
from dataloader import get_data, get_mini_data, load_json, save_json
from train import fit, custom_loss, validate
from metrics import calc_metrics

# paths
data_path = './dataset/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


uncomment the cell below if you want your experiments to yield always the same results

In [12]:
# manualSeed = 42

# np.random.seed(manualSeed)
# torch.manual_seed(manualSeed)

# # if you are using GPU
# torch.cuda.manual_seed(manualSeed)
# torch.cuda.manual_seed_all(manualSeed)

# torch.backends.cudnn.enabled = False 
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

#### Training

Initialize the model. Possible Values for the task block type: MLP, LSTM, GRU, TempConv

In [13]:
params = {'name': 'lstm', 'type_': 'LSTM', 'lr': 3e-4, 
          'n_h': 128, 'p':0.5, 'seq_len':3}

In [14]:
save_json(params, f"models/{params['name']}")

In [15]:
model, opt = get_model(params)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

get the data loader. get mini data gets only a subset of the training data, on which we can try if the model is able to overfit

In [16]:
train_dl, valid_dl = get_data(data_path, model.params.seq_len, batch_size=16)
# train_dl, valid_dl = get_mini_data(data_path, model.params.seq_len, batch_size=16, l=4000)

Train the model. We automatically save the model with the lowest val_loss. If you want to continue the training and keep the loss history, just pass it as an additional argument as shown below.

In [ ]:
model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl)

In [ ]:
# model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl, val_hist=val_hist)

uncomment the following two cells if the feature extractor should also be trained

In [ ]:
# for name,param in model.named_parameters():
#     param.requires_grad = True
# opt = optim.Adam(model.parameters())

In [ ]:
# model, val_hist = fit(1, model, custom_loss, opt, train_dl, valid_dl)

In [ ]:
plt.plot(val_hist)

#### evalute the model

In [17]:
name = 'gru'
params = load_json(f"models/{name}")

In [18]:
model, _ = get_model(params)
model.load_state_dict(torch.load(f"./models/{name}.pth"));

In [36]:
model.eval().to(device);
_, valid_dl = get_data(data_path, model.params.seq_len, batch_size=1)

In [41]:
_, all_preds, all_labels = validate(model, valid_dl, custom_loss)




  0%|          | 0/47802 [00:00<?, ?it/s]/home/rsi/repos/CAL/training/train.py:49: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(x.squeeze(), y)*w


RuntimeError: output with shape [1] doesn't match the broadcast shape [2]

In [ ]:
calc_metrics(all_preds, all_labels)

In [ ]:
# GRU: 
#  eval: 0.008826
# {'red_light_val_acc': 97.82017488807999,
#  'red_light_IoU': 84.50293740970099,
#  'hazard_stop_val_acc': 99.26990502489436,
#  'hazard_stop_IoU': 82.67147279218854,
#  'speed_sign_val_acc': 99.06489268231455,
#  'speed_sign_IoU': 86.63423493538414,
#  'relative_angle_MAE_mean': 0.11879964,
#  'center_distance_MAE_mean': 0.33218363,
#  'veh_distance_MAE_mean': 0.31114423}

#### plot results

In [ ]:
# for convience, we can pass an integer instead of the full string
int2key = {0: 'red_light', 1:'hazard_stop', 2:'speed_sign', 
           3:'relative_angle', 4: 'center_distance', 5: 'veh_distance'}

In [ ]:
def plot_preds(k, all_preds, all_labels, start=0, delta=1000):
    if isinstance(k, int): k = int2key[k]
    
    # get preds and labels
    class_labels = ['red_light', 'hazard_stop', 'speed_sign']
    pred = np.argmax(all_preds[k], axis=1) if k in class_labels else all_preds[k]
    label = all_labels[k][:, 1] if k in class_labels else all_labels[k]
    
    plt.plot(pred[start:start+delta], 'r--', label='Prediction', linewidth=2.0)
    plt.plot(label[start:start+delta], 'g', label='Ground Truth', linewidth=2.0)
    
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_preds(5, all_preds, all_labels, start=0, delta=4000)